In [9]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline 

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder 

from sklearn.impute import SimpleImputer 

from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline 

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split, GridSearchCV


In [10]:
train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')

In [11]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
# To get some info about the object colomns 
train_df.describe(include=['O'])

In [ ]:
train_df.gr oupby(['Pclass'], as_index=False)['Survived'].mean()

You have a higher chance to survive if you are in the 1st class

In [ ]:
train_df.groupby(['Sex'], as_index=False)['Survived'].mean()

Even though there is more male then female, female are most likely to survive than male

In [ ]:
train_df.groupby(['SibSp'], as_index=False)['Survived'].mean()

In [ ]:
train_df.groupby(['Parch'], as_index=False)['Survived'].mean()

In [ ]:
train_df['Family_Size'] = train_df['SibSp'] + train_df['Parch'] + 1
test_df['Family_Size'] = test_df['SibSp'] + test_df['Parch'] + 1

To get the size of a familly 

In [ ]:
train_df.head()

In [ ]:
train_df.groupby(['Family_Size'], as_index=False)['Survived'].mean()

In [ ]:
family_map = {1: 'Alone', 2: 'Small', 3:'Small', 4:'Small', 5:'Medium', 6:'Medium', 7:'Large', 8:'Large'}
train_df['Family_Size_Grouped'] = train_df['Family_Size'].map(family_map)
test_df['Family_Size_Grouped'] = test_df['Family_Size'].map(family_map)

In [ ]:
train_df.groupby(['Family_Size_Grouped'], as_index=False)['Survived'].mean()

In [ ]:
train_df.groupby(['Embarked'], as_index=False)['Survived'].mean()